# Финальный проект по SQL

## Задача

Проанализировать базу данных крупного сервиса для чтения книг по подписке с информацией о книгах, издательствах, авторах, а также о пользовательских обзорах на книги. Эти данные помогут сформулировать ценностное предложение для нового продукта.

## Описание данных

**Таблица `books`** содержит информацию о книгах и состоит из следующих полей:

- `book_id` — идентификатор книги;
- `author_id` — идентификатор автора;
- `title` — название книги;
- `num_pages` — количество страниц;
- `publication_date` — дата публикации книги;
- `publisher_id` — идентификатор издателя.

**Таблица `authors`** содержит информацию об авторах и состоит из следующих полей:

- `author_id` — идентификатор автора;
- `author` — имя автора.

**Таблица `publishers`** содержит информацию об издательствах и состоит из следующих полей:

- `publisher_id` — идентификатор издательства;
- `publisher` — название издательства;

**Таблица `ratings`** содержит информацию о пользовательских оценках книг и состоит из следующих полей:

- `rating_id` — идентификатор оценки;
- `book_id` — идентификатор книги;
- `username` — имя пользователя, оставившего оценку;
- `rating` — оценка книги.

**Таблица `reviews`** содержит информацию о пользовательских обзорах на книги и состоит из следующих полей:

- `review_id` — идентификатор обзора;
- `book_id` — идентификатор книги;
- `username` — имя пользователя, написавшего обзор;
- `text` — текст обзора.

# Содержание

1. [Сколько книг вышло после 1 января 2000 года](#1)
2. [Количество обзоров и средняя оценка каждой книги](#2)
3. [Издательство, которое выпустило наибольшее число книг толще 50 страниц (исключаем из анализа брошюры)](#3)
4. [Автор с самой высокой средней оценкой книг (учитываем только книги с 50 и более оценками)](#4)
5. [Среднее количество обзоров от пользователей, которые поставили больше 50 оценок](#5)
6. [Выводы](#6)

In [1]:
# импортируем библиотеки
import pandas as pd
from sqlalchemy import create_engine

# устанавливаем параметры

db_config = {
    'user': 'praktikum_student', # имя пользователя
    'pwd': 'Sdf4$2;d-d30pp', # пароль
    'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
    'port': 6432, # порт подключения
    'db': 'data-analyst-final-project-db'} # название базы данных
connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
                                                         db_config['pwd'],
                                                         db_config['host'],
                                                         db_config['port'],
                                                         db_config['db'])
# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'}) 


In [2]:
def sql(query):
    dataframe = pd.io.sql.read_sql(query, con = engine)
    return dataframe

In [3]:
books = '''
            SELECT * 
            FROM books
            LIMIT 5
        '''
sql(books)

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


In [4]:
authors = '''
            SELECT * 
            FROM authors
            LIMIT 5
        '''
sql(authors)

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


In [5]:
publishers = '''
            SELECT * 
            FROM publishers
            LIMIT 5
        '''
sql(publishers)

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


In [6]:
ratings = '''
            SELECT * 
            FROM ratings
            LIMIT 5
        '''
sql(ratings)

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


In [7]:
reviews = '''
            SELECT * 
            FROM reviews
            LIMIT 5
        '''
sql(reviews)

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


# 1. Сколько книг вышло после 1 января 2000 года<a id="1"></a>

In [8]:
query1 = '''
            SELECT
                COUNT(title) AS cnt
            FROM
                books
            WHERE
                publication_date > '2000-01-01'
        '''
sql(query1)

,cnt
0,819


После 1 января 2000 года (не включительно) вышли 819 книг

# 2. Количество обзоров и средняя оценка каждой книги<a id="2"></a>

In [16]:
# Делаем запрос, сортируя по количеству обзоров по убыванию
query21 = '''
            SELECT
                books.book_id AS book_id,
                books.title AS title,
                COUNT(DISTINCT review_id) as cnt_reviews,
                AVG(rating) as avg_rating
            FROM
                books 
                FULL JOIN reviews 
                    ON books.book_id = reviews.book_id
                FULL JOIN ratings 
                    ON books.book_id = ratings.book_id
            GROUP BY
                books.book_id
            ORDER BY 
                cnt_reviews DESC
            LIMIT 10

         '''
sql(query21)

,book_id,title,cnt_reviews,avg_rating
0,948,Twilight (Twilight #1),7,3.662500
1,696,The Da Vinci Code (Robert Langdon #2),6,3.830508
2,207,Eat Pray Love,6,3.395833
3,627,The Alchemist,6,3.789474
4,673,The Catcher in the Rye,6,3.825581
5,695,The Curious Incident of the Dog in the Night-Time,6,4.081081
6,302,Harry Potter and the Prisoner of Azkaban (Harr...,6,4.414634
7,299,Harry Potter and the Chamber of Secrets (Harry...,6,4.287500
8,497,Outlander (Outlander #1),6,4.125000
9,656,The Book Thief,6,4.264151


In [10]:
# Делаем запрос, сортируя по количеству обзоров по возрастанию
query22 = '''
            SELECT
                books.book_id AS book_id,
                books.title AS title,
                COUNT(DISTINCT review_id) as cnt_reviews,
                AVG(rating) as avg_rating
            FROM
                books 
                FULL JOIN reviews 
                    ON books.book_id = reviews.book_id
                FULL JOIN ratings 
                    ON books.book_id = ratings.book_id
            GROUP BY
                books.book_id
            ORDER BY 
                cnt_reviews
            LIMIT 10

         '''
sql(query22)

,book_id,title,cnt_reviews,avg_rating
0,808,The Natural Way to Draw,0,3.000000
1,191,Disney's Beauty and the Beast (A Little Golden...,0,4.000000
2,221,Essential Tales and Poems,0,4.000000
3,672,The Cat in the Hat and Other Dr. Seuss Favorites,0,5.000000
4,387,Leonardo's Notebooks,0,4.000000
5,83,Anne Rice's The Vampire Lestat: A Graphic Novel,0,3.666667
6,186,Death: The High Cost of Living,1,3.000000
7,165,Creepshow,1,4.500000
8,187,Debt of Honor (Jack Ryan #7),1,3.000000
9,14,A Christmas Carol and Other Christmas Writings,1,4.500000


In [11]:
# Делаем запрос, сортируя по рейтингу по возрастанию
query23 = '''
            SELECT
                books.book_id AS book_id,
                books.title AS title,
                COUNT(DISTINCT review_id) as cnt_reviews,
                AVG(rating) as avg_rating
            FROM
                books 
                FULL JOIN reviews 
                    ON books.book_id = reviews.book_id
                FULL JOIN ratings 
                    ON books.book_id = ratings.book_id
            GROUP BY
                books.book_id
            ORDER BY 
                avg_rating
            LIMIT 10 

         '''
sql(query23)

,book_id,title,cnt_reviews,avg_rating
0,303,Harvesting the Heart,2,1.500000
1,371,Junky,2,2.000000
2,202,Drowning Ruth,3,2.000000
3,316,His Excellency: George Washington,2,2.000000
4,915,The World Is Flat: A Brief History of the Twen...,3,2.250000
5,794,The Mermaid Chair,3,2.333333
6,772,The Kitchen God's Wife,3,2.333333
7,313,Heretics of Dune (Dune Chronicles #5),2,2.500000
8,72,Amsterdam,2,2.500000
9,193,Dr. Faustus,2,2.500000


Больше всего обзоров на книги у книги Twilight  - 7 шт (средний рейтинг - 3.7)

0 обзоров на книги у 6 следующих книг:
- The Natural Way to Draw
- Disney's Beauty and the Beast 
- Essential Tales and Poems
- The Cat in the Hat and Other Dr. Seuss Favorites
- Leonardo's Notebooks	
- Anne Rice's The Vampire Lestat: A Graphic Novel

Ниже всего рейтинг (1.5) у книги Harvesting the Heart

# 3. Издательство, которое выпустило наибольшее число книг толще 50 страниц (исключаем из анализа брошюры)<a id="3"></a>

In [12]:
query3 = '''
            SELECT
                publishers.publisher AS publisher,
                COUNT(book_id) as cnt_books
            FROM
                publishers 
                FULL JOIN books 
                    ON publishers.publisher_id = books.publisher_id
            WHERE
                books.num_pages > 50
            GROUP BY
                publishers.publisher
            ORDER BY 
                cnt_books DESC
            LIMIT 10 

         '''
sql(query3)

,publisher,cnt_books
0,Penguin Books,42
1,Vintage,31
2,Grand Central Publishing,25
3,Penguin Classics,24
4,Ballantine Books,19
5,Bantam,19
6,Berkley,17
7,St. Martin's Press,14
8,Berkley Books,14
9,William Morrow Paperbacks,13


Издательство, которое выпустило наибольшее число книг толще 50 страниц - Penguin Books (42 книги)

Топ-3 издательства, которые выпустили наибольшее количество книг:
- Penguin Books (42 книги)
- Vintage (31 книга)
- Grand Central Publishing	(25 книг)

# 4. Автор с самой высокой средней оценкой книг (учитываем только книги с 50 и более оценками)<a id="4"></a>

In [13]:
query4 = '''
              SELECT
                  authors.author AS author,
                  AVG(SUBQ.avg_rating) as avg_author_rating
              FROM
                  authors
                  INNER JOIN (
                        SELECT
                            books.book_id AS book_id,
                            books.author_id AS author_id,
                            AVG(rating) AS avg_rating,
                            COUNT(rating) AS cnt_rating
                        FROM   
                            books 
                            INNER JOIN ratings
                            ON books.book_id = ratings.book_id
                        GROUP BY
                            books.book_id
                        ORDER BY 
                            cnt_rating DESC) AS SUBQ ON authors.author_id = SUBQ.author_id

              WHERE
                  SUBQ.cnt_rating >= 50
              GROUP BY
                  authors.author
              ORDER BY 
                  avg_author_rating DESC
              LIMIT 10
                
         '''
sql(query4)

,author,avg_author_rating
0,J.K. Rowling/Mary GrandPré,4.283844
1,Markus Zusak/Cao Xuân Việt Khương,4.264151
2,J.R.R. Tolkien,4.258446
3,Louisa May Alcott,4.192308
4,Rick Riordan,4.080645
5,William Golding,3.901408
6,J.D. Salinger,3.825581
7,Paulo Coelho/Alan R. Clarke/Özdemir İnce,3.789474
8,William Shakespeare/Paul Werstine/Barbara A. M...,3.787879
9,Dan Brown,3.754540


- Самая высокая средняя оценка книг у J.K. Rowling, которая также пишет под псевдонимом Mary GrandPré	 (4.28)
- На втором месте по средней оценке книг - Markus Zusak/Cao Xuân Việt Khương со средним рейтингом 4.26
- На третьем месте - J.R.R. Tolkien (рейтинг - 4.258)

# 5. Среднее количество обзоров от пользователей, которые поставили больше 50 оценок<a id="5"></a>

In [14]:
query5 = '''
              SELECT
                  AVG(SUBQ1.cnt_review) as avg_review
              FROM (
                  SELECT
                      reviews.username AS username,
                      COUNT(review_id) AS cnt_review
                  FROM   
                      reviews
                  GROUP BY
                      reviews.username
                  ORDER BY 
                      cnt_review DESC) AS SUBQ1
                        
                  RIGHT JOIN (
                            SELECT
                                ratings.username AS username,
                                COUNT(rating) AS cnt_rating
                            FROM   
                                ratings
                            GROUP BY
                                ratings.username
                            HAVING
                                COUNT(rating) > 50
                            ORDER BY 
                                cnt_rating DESC
                            
                    ) AS SUBQ2 ON SUBQ1.username = SUBQ2.username
                
         '''
sql(query5)

,avg_review
0,24.333333


Пользователи, которые поставили больше 50 оценок, в среднем пишут около 24 обзоров

# 6. Выводы<a id="6"></a>

После 1 января 2000 года (не включительно) вышло **819** книг


Больше всего обзоров у книги **Twilight**  - 7 шт (средний рейтинг книги - 3.7)


Топ-3 издательства, которые выпустили наибольшее количество книг:
- **Penguin Books (42 книги)**
- Vintage (31 книга)
- Grand Central Publishing	(25 книг)
    

Топ-3 автора с самой высокой средней оценкой:
- **J.K. Rowling, которая также пишет под псевдонимом Mary GrandPré (рейтинг - 4.28)**
- Markus Zusak, который также пишет под именем Cao Xuân Việt Khương (рейтинг - 4.26)
- J.R.R. Tolkien (рейтинг - 4.258)
 
Пользователи, которые поставили больше 50 оценок, в среднем пишут около **24 обзоров**